In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.getcwd() 
import warnings
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import T5Tokenizer,T5Model,T5ForConditionalGeneration
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from utils import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *

In [11]:
parser = argparse.ArgumentParser("main")


parser.add_argument('--valid_num_points', type=int,             default = 100, help='validation data number')
parser.add_argument('--train_num_points', type=int,             default = 1000, help='train data number')

parser.add_argument('--batch_size', type=int,                   default=24,     help='Batch size')

parser.add_argument('--gpu', type=int,                          default=0,      help='gpu device id')
parser.add_argument('--G_model_name', type=str,                 default='t5-small',      help='model_name')
parser.add_argument('--D_model_name', type=str,                 default='t5-small',      help='model_name')
parser.add_argument('--exp_name', type=str,                     default='T5spec',      help='experiment name')
parser.add_argument('--rep_num', type=int,                      default=25,      help='report times for 1 epoch')
parser.add_argument('--test_num', type=int,                     default=4,      help='test times for 1 epoch')

parser.add_argument('--epochs', type=int,                       default=50,     help='num of training epochs')
parser.add_argument('--pre_epochs', type=int,                   default=0,      help='train model W for x epoch first')
parser.add_argument('--grad_clip', type=float,                  default=1,      help='gradient clipping')
parser.add_argument('--grad_acc_count', type=float,             default=-1,      help='gradient accumulate steps')

parser.add_argument('--G_lr', type=float,                       default=0.001,   help='learning rate for G')
parser.add_argument('--D_lr', type=float,                       default=0.001,   help='learning rate for D')
parser.add_argument('--lambda_identity', type=float,            default=0.5,   help='')
parser.add_argument('--lambda_A', type=float,                   default=10.0,   help='')
parser.add_argument('--lambda_B', type=float,                   default=10.0,   help='')
parser.add_argument('--learning_rate_min', type=float,          default=1e-8,   help='learning_rate_min')
parser.add_argument('--decay', type=float,                      default=1e-3,   help='weight decay')
parser.add_argument('--momentum', type=float,                   default=0.7,    help='momentum')
parser.add_argument('--smoothing', type=float,                  default=0.1,    help='labelsmoothing')


parser.add_argument('--traindata_loss_ratio', type=float,       default=0.9,    help='human translated data ratio')
parser.add_argument('--syndata_loss_ratio', type=float,         default=0.1,    help='augmented dataset ratio')

parser.add_argument('--valid_begin', type=int,                  default=1,      help='whether valid before train')
parser.add_argument('--train_A', type=int,                      default=1 ,     help='whether train A')


args = parser.parse_args(args=[])#(args=['--batch_size', '8',  '--no_cuda'])#used in ipynb

In [12]:
Gmodelname = args.G_model_name
Dmodelname = args.D_model_name
Gpretrained  =  T5ForConditionalGeneration.from_pretrained(Gmodelname)
Dpretrained  =  T5ForConditionalGeneration.from_pretrained(Dmodelname)
Gpathname = Gmodelname.replace('/','')
Dpathname = Dmodelname.replace('/','')
print(f'Gmodelsize:{count_parameters_in_MB(Gpretrained)}MB')
print(f'Dmodelsize:{count_parameters_in_MB(Dpretrained)}MB')
torch.save(Gpretrained,Gpathname+'.pt')
torch.save(Dpretrained,Dpathname+'.pt')


tokenizer = T5Tokenizer.from_pretrained(Gmodelname)#TODO:its wrong if tokenizer are diff for D and G

Gmodelsize:60.506624MB
Dmodelsize:60.506624MB


In [ ]:
cycleGAN = CycleGAN(args=args)